In [ ]:
!pip install -q gradio torch accelerate

In [ ]:
from transformers import pipeline
import gradio as gr

generator= pipeline(
    "text2text-generation",
    model="MBZUAI/LaMini-flan-T5-783M",
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
def generate_idea(domain):
    if not domain or not domain.strip():
      return "Please enter a domain (e.g Agritech, Fintech, Edtech)."
    prompt=f"""
      Generate a startup pitch for the domain:{domain},
      Include the following sections:
      1) Business Idea (2-3 sentences)
      2)Problem Statement (explain the challenge clearly)
      3)Solution (explain how the startup is solving it)
      """
    result=generator(
      prompt,
      max_new_tokens=100,
      temperature=0.8,
      do_sample=True,
    )
    return result[0]["Generated_text"]

In [ ]:
demo=gr.Interface(
    fn=generate_idea,
    inputs=gr.Textbox(label="Domain of Interest",placeholder="e.g. Agritech"),
    outputs="text",
    title="Startup Idea + Pitch Generator",
    description="Enter a domain (Agriculture,Fintech,Edtech, etc.) and get a startup pitch with Problem and Solution ",
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03e9a18c4e3b300f59.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Part 2 ( Improved version)

In [1]:
# !pip install -q gradio transformers accelerate torch

import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import textwrap
import random

MODEL_ID = "MBZUAI/LaMini-flan-T5-783M"

def load_model(model_id=MODEL_ID):
    """
    Load tokenizer+model once. Use device_map='auto' if CUDA available to shard automatically.
    Returns (tokenizer, model, pipeline_instance)
    """
    print("Loading model:", model_id)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # If CUDA available, use accelerate sharding for better memory usage
    model_kwargs = {}
    if torch.cuda.is_available():
        model_kwargs["device_map"] = "auto"
        model_kwargs["torch_dtype"] = torch.float16

    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, **model_kwargs)

    accelerate_mapped = getattr(model, "hf_device_map", None) is not None

    if accelerate_mapped:
        gen = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
    else:
        device = 0 if torch.cuda.is_available() else -1
        gen = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device)

    print("Model loaded. accelerate_mapped =", bool(accelerate_mapped), "cuda_available =", torch.cuda.is_available())
    return tokenizer, model, gen

tokenizer, model, generator = load_model(MODEL_ID)


# Prompt builder (few-shot + structured output)
def build_prompt(domain: str):
    domain = domain.strip()
    # Few-shot examples help the model stick to the format.
    prompt = textwrap.dedent(f"""
    You are an assistant that writes concise startup pitches. Produce clear labeled sections exactly as listed below.
    Keep Business Idea to 2-3 sentences. Use plain language and concrete problem/solution statements.

    Example:
    Business Idea:
    A mobile-first marketplace connecting smallholder farmers with cold-storage logistics and bulk buyers for fresh produce. The platform uses simple messaging + SMS to manage pickups and payments.

    Problem Statement:
    Smallholder farmers lose up to 30% of harvest due to lack of timely logistics and market access, receiving unfair prices from intermediaries.

    Solution:
    Provide an aggregator service that schedules refrigerated pickups, offers price transparency, and matches shipments to regional buyers using simple mobile tools and revenue-share contracts.

    -- Now generate a new pitch for the domain: {domain}

    Output format (use these headings exactly):
    Business Idea:
    Problem Statement:
    Solution:
    Revenue Model:
    Go-to-Market (first 3 steps):
    Key Risks:
    One-line Tagline:
    """).strip()
    return prompt


def generate_idea(domain, temperature, max_new_tokens, do_sample, seed, domain_suggestion):

    selected_domain = (domain or domain_suggestion or "").strip()
    if not selected_domain:
        return "Please enter a domain (e.g. Agritech, Fintech, Edtech).", ""

    if len(selected_domain) > 200:
        return "Domain text too long — please enter a short domain name.", ""

    # Construct prompt
    prompt = build_prompt(selected_domain)

    if seed is not None and seed != "":
        try:
            seed = int(seed)
            torch.manual_seed(seed)
            random.seed(seed)
        except Exception:
            seed = None

    gen_kwargs = {
        "max_new_tokens": int(max_new_tokens),
        "do_sample": bool(do_sample),
    }
    if do_sample:
        gen_kwargs["temperature"] = float(temperature)
        gen_kwargs["top_p"] = 0.95
    else:
        # beam search fallback (deterministic)
        gen_kwargs["num_beams"] = 4
        gen_kwargs["temperature"] = 0.0

    try:
        raw = generator(prompt, **gen_kwargs)
        text = raw[0].get("generated_text") or raw[0].get("text") or str(raw[0])
        formatted = text.strip()
    except Exception as e:
        formatted = f"Generation failed: {e}"
        text = formatted

    return formatted, text


with gr.Blocks(theme=gr.themes.Default()) as demo:
    gr.Markdown("# 🚀 Startup Idea & Pitch Generator")
    with gr.Row():
        with gr.Column(scale=2):
            domain_input = gr.Textbox(label="Domain (type)", placeholder="e.g. Agritech, Fintech, Edtech", lines=1)
            domain_dropdown = gr.Dropdown(
                label="Or choose a domain",
                choices=["Agritech", "Fintech", "Edtech", "Healthtech", "ClimateTech", "Logistics", "SaaS", "Marketplace"],
                value="Agritech"
            )
            with gr.Row():
                temp_slider = gr.Slider(minimum=0.0, maximum=1.2, value=0.8, label="Temperature (creativity)")
                tokens_slider = gr.Slider(minimum=32, maximum=800, value=220, step=1, label="Max new tokens")
            with gr.Row():
                sample_checkbox = gr.Checkbox(label="Use sampling (more creative)", value=True)
                seed_input = gr.Number(label="Seed (optional, integer)", value=None)
            gen_btn = gr.Button("Generate Pitch")
            examples = gr.Examples(
                examples=[["Agritech"], ["Fintech"], ["Edtech"]],
                inputs=[domain_input],
                fn=None
            )

        with gr.Column(scale=3):
            out_md = gr.Markdown("### Generated Pitch")
            out_text = gr.Textbox(label="Pitch (plain text)", lines=15)
            raw_section = gr.Accordion("Raw generator output", open=False)
            with raw_section:
                raw_text = gr.Textbox(label="Raw text", lines=6)

    # Wire the button
    gen_btn.click(
        fn=generate_idea,
        inputs=[domain_input, temp_slider, tokens_slider, sample_checkbox, seed_input, domain_dropdown],
        outputs=[out_text, raw_text]
    )

    gr.Markdown(
        """
        **Tips**
        - Use sampling for creative idea generation; turn it off for a concise, deterministic answer.
        - Increase max tokens if you want longer, more detailed plans.
        - Use the seed to reproduce results.
        """
    )

# Launch the app
demo.launch(share=True)


Loading model: MBZUAI/LaMini-flan-T5-783M


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


Model loaded. accelerate_mapped = True cuda_available = True
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f3da4899fea42d6e7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
